In [ ]:
# Alpcan YILDIZ 220201049

In [ ]:
import cv2
import numpy as np
import math
from numpy.linalg import inv # To compute the inverse of Homography
from math import floor

In [ ]:
def initializeAffine(ψ,θ,λ):
    affine=λ*np.array([[math.cos(np.deg2rad(ψ)), math.sin(np.deg2rad(ψ))], [-math.sin(np.deg2rad(ψ)), math.cos(np.deg2rad(ψ))]])
    return np.dot(affine,np.array([[math.cos(np.deg2rad(θ)), 0], [0, 1]]))

#In-plane rotation angle (ψ) represents rotation around an axis that is perpendicular to the object plane

#Tilt amount (θ) is the angle between the normal of the image plane and the optical axis of the camera

#Scale (λ) is the zoom parameter

def findWarpedImageSize(affine,h,w):  #To find the warped image size. Multiplying by affine matrix with 4 corners of the reference image
    topleftCorner = np.dot(affine,np.array([[0],[0]]))
    toprightCorner = np.dot(affine,np.array([[h],[0]]))
    bottomleftCorner = np.dot(affine,np.array([[0],[w]]))
    bottomrightCorner = np.dot(affine,np.array([[h],[w]]))
    
    h=abs(round(int(topleftCorner[0] - bottomrightCorner[0])))
    w=abs(round(int(bottomleftCorner[1]-toprightCorner[1]))) 
    
    return np.zeros([h,w,1]) #Create image with new size

def findHomogrophy(affine,ww,wh,rw,rh): #To find homogrophy matrix
    first=np.dot(np.array([[1,0,ww/2],[0,1,wh/2],[0,0,1]]),np.array([[affine[0][0],affine[0][1],0],[affine[1][0],affine[1][1],0],[0,0,1]]))
    return np.dot(first,np.array([[1,0,(-rh/2)],[0,1,(-rw/2)],[0,0,1]]))


def visitPixels(referenceImg,warpedImg): #Visiting warped image pixels
    x=0
    for row in warpedImg:
        y=0
        for pixel in row:
            findpointsWarpedToReference(referenceImg,warpedImg,x,y)
            y+=1
       # print ("loading:",(x/len(warpedImg))*100)
        x+=1
        
def findpointsWarpedToReference(referenceImg,warpedImg,x,y): #From warped image pixels to reference image we can go via inverse of homogrophy
    temp = np.dot(h_inv,np.array([[x],[y],[1]]))  
    if (temp[0] < 0 or temp[0] >= len(referenceImg)-1): #This is for warped image pixels that does not match with the reference image pixel. It just pass and make those pixels black.
        warpedImg[x][y] = 0
        return False
    if (temp[1] < 0 or temp[1] >= len(referenceImg[0])-1): #This is for warped image pixels that does not match with the reference image pixel. It just pass and make those pixels black.
        warpedImg[x][y] = 0
        return False
    else:      
        warpedImg[x][y]=bilinearInterpolation(referenceImg,temp[0],temp[1]) #If it is matched then apply the Bilinear Interpolation
    
def bilinearInterpolation(img,x,y):  

    roundedx=round(float(x)) #First I round x and y values
    roundedy=round(float(y))
    floorx = (x-roundedx)  # Then substract with rounded values
    floory = (y-roundedy)
    
    if(floorx>0):       #There are x and y distance conditions which our pixels belongs to true inside the 4 pixel. 
        if(floory>0):
           
            a0= (1 - floorx) * (1-floory) * img[roundedx][roundedy]
            a1= floorx * (1-floory) * img[roundedx+1][roundedy]
            a2= (1 - floorx)*floory *img[roundedx][roundedy+1] 
            a3= floorx * floory * img[roundedx+1][roundedy+1]
        elif (floory<0):
            
            floory =abs(floory)
            
            a0= (1 - floorx)  * (1-floory)* img[roundedx][roundedy]
            a1=  floorx * (1-floory)*img[roundedx+1][roundedy]
            a2=  (1 - floorx)* floory  *img[roundedx][roundedy-1]
            a3= floorx * floory * img[roundedx-1][roundedy-1]
    
    elif(floorx<0):
        
        floorx = abs(floorx)
        
        if(floory>0):
           
            a0=  (1 - floorx) * img[roundedx][roundedy] * (1-floory)
            a1=   floorx* (1-floory) * img[roundedx-1][roundedy]
            a2= (1 - floorx)*floory *  img[roundedx][roundedy+1] 
            a3= floorx * floory * img[roundedx-1][roundedy+1]
        elif (floory<0):
           
            floory =abs(floory)
            a0=(1 - floorx)  * (1-floory) * img[roundedx][roundedy]
            a1=floorx* (1-floory)* img[roundedx-1][roundedy]
            a2=(1 - floorx)*floory *img[roundedx][roundedy-1] 
            a3=floorx * floory * img[roundedx-1][roundedy-1]
    
    elif(floorx==0 or floory==0): #If it is in the middle
       
        a0= 0,25 *img[roundedx][roundedy] 
        a1= 0,25 *img[roundedx+1][roundedy]
        a2= 0,25 *img[roundedx][roundedy+1]
        a3= 0,25 *img[roundedx+1][roundedy+1]

    intensity = (a0 + a1 + a2 + a3)             
    return floor(intensity)
 
def nearestInterpolation(img,x,y):

    roundedx=round(float(x))
    roundedy=round(float(y))
         
    return img[roundedx][roundedy]

In [ ]:
img1 = cv2.imread("img1.png",0)    #Read image
h1,w1 = img1.shape
affine=initializeAffine(30,50,0.5)  #Initialize the affine matrix.

print ("This is affine matrix:", affine)

imgWarped=findWarpedImageSize(affine,h1,w1)   #Find size for warped image and create black image
homogropyh = findHomogrophy(affine,len(imgWarped),len(imgWarped[0]),w1,h1) #Find homogropghy

print ("homogropyh",homogropyh)
h_inv = inv(homogropyh)  #Find inverse of homogrophy

print ("homogropyh INV",h_inv)  

visitPixels(img1,imgWarped) #Call the main function so that it call anothers functions to complete the task

cv2.imwrite("img_warped.png",imgWarped) #Save the warped image

#  COMMENT 1

1)--- We multiply two matrices with the homogrophy matrix to set relationship between the images with their widht-heiht.
2)--- We can just use the homogrophy matrix but this time warped image DOES NOT fit the image we create with the corners of the reference image. Also inverse matrix changes when we multiply or we dont.
I tried with 3 differents way and saved the images;

# Observation
IN the file there 3 images with their names below conditions
1----> I just used homogrophy matrix and runned the code
2----> I used first matrix with homogrophy matrix and runned the code 
3----> I used homogrophy matrix with third matrix and runned the code
 In both of them our warped image did not fit. Images that I tried, they are like cropped or slides from the center. It was like we used the translation and move the pixels to right / left / up / and down.
 We could not get the image entirely.
 So, to create a relationship between them we multiply those matrices with each other with homogropyh from its left and right side. We use both of them image warped just fit the image.
 
 And I noticed that 2. and 3. conditions they are related like : firstMatrix X Homogrophy and Homogrophy X Third Matrix
 they are the continuing of their selves. (Birbirinin devamları)
 If we merge two pictures they belong one picture but ofcourse they are not fit, it is away from center and not normal.

# COMMENT 2
1) We find the corresponding points that warped image pixels to reference image pixels via inverse of the homogropyh matrix. Points may come fractional. We can go the homogrophy matrix. This time we lose some pixels from reference to warped image because of scale. For example, 4 pixels corrensponds 1 pixels but sometimes it does not correspond 4 pixels at all because tilt and rotation. (We can take the average intensity of the pixels) Let's not forget about that our Scale (λ) was 0.5. What if our scale was > 1. Then we can't go to reference image to warped image. We dont have enough pixels to do that. How we can fit small images pixels to big image pixels ? It will be 1 pixels corresponds more pixels. To get a certain result and not to face with the problem, we go back with the inverse homogrophy and find corresponding intensities.

# COMMENT 3
In the file, there is an image named "img_warped_nearestInterpolation.png "
---- I defined a function " nearestInterpolation " and runs the code with it. It gave me same image but bad result. It is like our pixels shaked. Because in the bilinear interpolation we use relation that x,y distances to pixels and use 4 pixels intensity. In this nearest interpolation we just round the nearest pixel and just use 1 pixel intensity. If we think that it is not logical at all to wait good result comparing bilinear interpolation.

